# Aggregration of Experiment Results

This notebook pulls results from the `output` folder and aggregates them in a large table that can than be formated, pivoted etc.

In [1]:
from pathlib import Path

OUTPUT_PATH = Path("/work/smt4/thulke/vieweg/SER/Code/sisyphus/output")

## Metrics

In [2]:
import pandas as pd
import glob

dfs = []
for path in map(Path, glob.glob(f"{OUTPUT_PATH}/**/metrics.csv", recursive=True)):
    if "pp" in path.parts:
        continue

    print(path)
    metrics_df = pd.read_csv(path)
    metrics_df["experiment"] = path.parts[-3]
    metrics_df["dataset"] = path.parts[-4]
    dfs.append(metrics_df)

metrics = pd.concat(dfs, axis=0, ignore_index=True)
metrics.head(20)

/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_finetune_facebook_wav2vec2-base-960h/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-53-english/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_featureextract_facebook_wav2vec2-base-960h/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/yuan/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/rav/lj_featureextract_jonatasgrosman_wav2vec2-large-xlsr-53-english/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/iem/lj_featureextract_facebook_wav2vec2-base-960h/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/iem/lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-53-english/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/iem/yuan/results/metrics.csv
/work/smt4/thulke/vieweg/SER/Code/sisyphus/output/iem/lj_finetune_facebook_wav2vec2-ba

,split,metric,value,experiment,dataset
0,train,eer,0.708333,lj_finetune_facebook_wav2vec2-base-960h,rav
1,val,eer,0.716667,lj_finetune_facebook_wav2vec2-base-960h,rav
2,test,eer,0.730000,lj_finetune_facebook_wav2vec2-base-960h,rav
3,train,beer_Happiness,0.958333,lj_finetune_facebook_wav2vec2-base-960h,rav
4,train,beer_Anger,0.812500,lj_finetune_facebook_wav2vec2-base-960h,rav
5,train,beer_Sadness,0.958333,lj_finetune_facebook_wav2vec2-base-960h,rav
6,train,beer_Fear,0.159722,lj_finetune_facebook_wav2vec2-base-960h,rav
7,train,beer_Disgust,0.062500,lj_finetune_facebook_wav2vec2-base-960h,rav
8,train,beer_Surprise,0.888889,lj_finetune_facebook_wav2vec2-base-960h,rav
9,train,beer_Calmness,0.972222,lj_finetune_facebook_wav2vec2-base-960h,rav


In [3]:
from IPython.display import Markdown

for split in metrics.split.unique():
    # global metrics
    global_only_metrics = metrics.loc[
        ~metrics.metric.str.contains("_") ^ metrics.metric.str.contains("total")
    ]
    global_only_metrics = global_only_metrics.assign(
        metric=global_only_metrics.metric.apply(lambda s: s.split("_")[0])
    )

    global_pivot_metrics = (
        global_only_metrics.query("split == @split").pivot(
            index="experiment", columns=["dataset", "metric"], values="value"
        )
        * 100
    )

    if split == "test":
        global_pivot_metrics.loc["lj_finetune_lit", ("rav", "eer")] = 18.01
        global_pivot_metrics.loc["lj_featureextract_lit", ("rav", "eer")] = 43.47
        global_pivot_metrics.loc["yuan_lit", ("iem", "beer")] = 100 - 75.6
        global_pivot_metrics.loc["yuan_lit", ("iem", "eer")] = 100 - 75.5

    # beer per emo
    per_emo_metrics = metrics.loc[
        metrics.metric.str.contains("_") & ~metrics.metric.str.contains("total")
    ]
    per_emo_metrics = per_emo_metrics.assign(
        emotion=per_emo_metrics.metric.apply(lambda s: s.split("_")[-1])
    )

    per_emo_pivot_metrics = (
        per_emo_metrics.query("split == @split").pivot(
            index="experiment", columns=["dataset", "emotion"], values="value"
        )
        * 100
    )

    # print
    display(Markdown(f"# {split}"))
    display(global_pivot_metrics.round(2).fillna("--"))
    display(per_emo_pivot_metrics.round(2).fillna("--"))

# train

dataset                                               rav           iem       
metric                                                eer   beer    eer   beer
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h       58.06  60.68  48.05  53.28
lj_featureextract_jonatasgrosman_wav2vec2-large...  58.06  60.59  44.30  47.30
lj_finetune_facebook_wav2vec2-base-960h             70.83  72.66  48.37  49.76
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  41.85  45.49  26.86  29.16
yuan                                                89.07  89.06  76.06  76.27

dataset                                                  rav                 \
emotion                                            Happiness  Anger Sadness   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h          93.06  40.28   75.69   
lj_featureextract_jonatasgrosman_wav2vec2-large...     78.47  34.03   80.56   
lj_finetune_facebook_wav2vec2-base-960h                95.83  81.25   95.83   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...     90.97   7.64   70.83   
yuan                                                   91.67  90.28   94.44   

dataset                                                                     \
emotion                                              Fear Disgust Surprise   
experiment                                                                   
lj_featureextract_facebook_wav2vec2-base-960h       33.33   86.11    42.36   
lj_featureextract_jonatasgrosman_wav2vec2-large...  58.33   56.25    61.81   
lj_finetune_facebook_wav2vec2-base-960h             15.97    6.25    88.89   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  30.56   26.39    31.25   
yuan                                                90.97   90.97    81.25   

dataset                                                                   iem  \
emotion                                            Calmness Neutral Happiness   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h         14.58  100.00     72.25   
lj_featureextract_jonatasgrosman_wav2vec2-large...    16.67   98.61     63.16   
lj_finetune_facebook_wav2vec2-base-960h               97.22  100.00     89.64   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...     6.25  100.00     38.90   
yuan                                                  84.03   88.89     75.00   

dataset                                                                    
emotion                                             Anger Sadness Neutral  
experiment                                                                 
lj_featureextract_facebook_wav2vec2-base-960h       18.18   85.27   37.45  
lj_featureextract_jonatasgrosman_wav2vec2-large...  22.50   64.35   39.21  
lj_finetune_facebook_wav2vec2-base-960h             11.02   32.43   65.95  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   6.01   38.74   32.97  
yuan                                                75.34   76.96   77.78

# val

dataset                                               rav           iem       
metric                                                eer   beer    eer   beer
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h       61.67  64.06  50.96  53.17
lj_featureextract_jonatasgrosman_wav2vec2-large...  60.00  62.50  46.15  49.78
lj_finetune_facebook_wav2vec2-base-960h             71.67  73.44  45.19  48.08
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  46.67  50.00  33.65  36.98
yuan                                                90.00  90.62  76.00  74.99

dataset                                                  rav                 \
emotion                                            Happiness  Anger Sadness   
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h          100.0   25.0    62.5   
lj_featureextract_jonatasgrosman_wav2vec2-large...      75.0   25.0    75.0   
lj_finetune_facebook_wav2vec2-base-960h                100.0   75.0   100.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...     100.0   12.5    62.5   
yuan                                                    75.0  100.0   100.0   

dataset                                                                     \
emotion                                              Fear Disgust Surprise   
experiment                                                                   
lj_featureextract_facebook_wav2vec2-base-960h        37.5   100.0     87.5   
lj_featureextract_jonatasgrosman_wav2vec2-large...   62.5    62.5    100.0   
lj_finetune_facebook_wav2vec2-base-960h              12.5     0.0    100.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...   50.0    12.5     50.0   
yuan                                                100.0   100.0     87.5   

dataset                                                                   iem  \
emotion                                            Calmness Neutral Happiness   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h           0.0   100.0     63.64   
lj_featureextract_jonatasgrosman_wav2vec2-large...      0.0   100.0     65.38   
lj_finetune_facebook_wav2vec2-base-960h               100.0   100.0     87.04   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...     12.5   100.0     53.70   
yuan                                                   62.5   100.0     76.19   

dataset                                                                    
emotion                                             Anger Sadness Neutral  
experiment                                                                 
lj_featureextract_facebook_wav2vec2-base-960h       16.95   93.33   38.78  
lj_featureextract_jonatasgrosman_wav2vec2-large...  19.70   66.67   47.37  
lj_finetune_facebook_wav2vec2-base-960h              9.21   32.26   63.83  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  11.84   41.94   40.43  
yuan                                                80.65   78.12   65.00

# test

dataset                                               rav           iem       
metric                                                eer   beer    eer   beer
experiment                                                                    
lj_featureextract_facebook_wav2vec2-base-960h       58.33  60.94  55.49  54.94
lj_featureextract_jonatasgrosman_wav2vec2-large...   64.0  65.62  51.18  50.81
lj_finetune_facebook_wav2vec2-base-960h              73.0  74.69  55.29  54.82
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  51.33  54.37  35.29  35.41
yuan                                                89.33  88.75  78.18  78.55
lj_finetune_lit                                     18.01     --     --     --
lj_featureextract_lit                               43.47     --     --     --
yuan_lit                                               --     --   24.5   24.4

dataset                                                  rav                \
emotion                                            Happiness Anger Sadness   
experiment                                                                   
lj_featureextract_facebook_wav2vec2-base-960h           97.5  17.5    75.0   
lj_featureextract_jonatasgrosman_wav2vec2-large...      80.0  20.0    90.0   
lj_finetune_facebook_wav2vec2-base-960h                 95.0  82.5    97.5   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...     100.0  32.5    82.5   
yuan                                                    90.0  95.0    92.5   

dataset                                                                    \
emotion                                             Fear Disgust Surprise   
experiment                                                                  
lj_featureextract_facebook_wav2vec2-base-960h       57.5    70.0     67.5   
lj_featureextract_jonatasgrosman_wav2vec2-large...  67.5    80.0     82.5   
lj_finetune_facebook_wav2vec2-base-960h             20.0    10.0     95.0   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  35.0    35.0     40.0   
yuan                                                95.0    92.5     82.5   

dataset                                                                   iem  \
emotion                                            Calmness Neutral Happiness   
experiment                                                                      
lj_featureextract_facebook_wav2vec2-base-960h           2.5   100.0     75.93   
lj_featureextract_jonatasgrosman_wav2vec2-large...     15.0    90.0     70.99   
lj_finetune_facebook_wav2vec2-base-960h                97.5   100.0     91.98   
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...     10.0   100.0     41.98   
yuan                                                   82.5    80.0     79.13   

dataset                                                                    
emotion                                             Anger Sadness Neutral  
experiment                                                                 
lj_featureextract_facebook_wav2vec2-base-960h       16.55   90.18   37.11  
lj_featureextract_jonatasgrosman_wav2vec2-large...  17.99   67.86   46.39  
lj_finetune_facebook_wav2vec2-base-960h              5.76   47.32   74.23  
lj_finetune_jonatasgrosman_wav2vec2-large-xlsr-...  20.86   43.75   35.05  
yuan                                                80.52   71.62   82.93